In [2]:
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [3]:
#get data for a given month as dataframe
def get_month_data(year: int, month: int, select_cols: str): 
    #get correct number of days based on month
    match month:
        case 2: 
            max_days = 28
        case 1 | 3 | 5 | 7 | 8 | 10 | 12:
            max_days = 31
        case 4 | 6 | 9 | 11:
            max_days = 30
    #write select statement given columns 
    statement = f"""SELECT {select_cols}
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, 
        UNNEST(hits) as hits LEFT JOIN UNNEST(product) as product LEFT JOIN UNNEST(promotion) AS promotion 
        WHERE _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', '{year}-{month}-01') AND FORMAT_DATE('%Y%m%d', '{year}-{month}-{max_days}')
        AND (eventInfo.eventAction IS NOT NULL OR eCommerceAction.action_type != '0')
        ORDER BY date, unique_id, hitNumber ASC;""" 
        #initialize bigquery connection
    client = bigquery.Client()
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(statement, job_config=job_config)
    results = query_job.result().to_dataframe()
    return(results)



In [4]:
#flatten JSON records
def flatten(df, *cols):
    for col in cols:
        temp = pd.json_normalize(df[col])
        print(col, len(temp.columns))
        df = pd.concat([df, temp], axis = 1)
        df = df.drop(col, axis = 1)
    return df 

In [5]:
#drop columns that are than 50% null or only have 1 unique value
def drop_columns(df):
    nulls = [None, '(not set)', '(not provided)', '(none)', '(not available in demo dataset)', 'not available in demo dataset', np.nan, [], '[]', ''] 
    #remove duplicates
    duplicated = df.columns[df.columns.duplicated()]
    for col in duplicated:
        temp = df[col].iloc[:,0]
        #drop both duplicates
        df = df.drop(col, axis = 1)
        #add back a single columns 
        df[col] = temp
    for col in (df.columns):
        print(col)
        #remove all nan only columns
        if pd.isna(df[col]).all() or df[col].isin(nulls).all():
            df = df.drop(col, axis =1)
        #remove columns where >50% of values are null: 
        elif (df[col].isin(nulls).value_counts()[False]/len(df.index) < 0.5):
            df = df.drop(col, axis = 1)
        #remove all cols that only have 1 unique value 
        elif (len(df[col].astype(str).unique()) <= 1):
            df = df.drop(col, axis =1)
    return df

In [8]:
#preprocessing pipeline
def preprocess(year, month, select_cols, *flatten_cols):
    month_df = get_month_data(year, month, select_cols)
    print("queried")
    flatten_df = flatten(month_df, *flatten_cols)
    print("flattened")
    #replace all empty arrays so they can be compared for null counts
    flatten_df = flatten_df.applymap(lambda v: '[]' if isinstance(v, np.ndarray) and len(v) == 0 else v)
    dropped_df = drop_columns(flatten_df)
    print("dropped columns")
    return(dropped_df)
def export(df, name):
    try:
        df.to_csv(f'{name}.csv', mode = 'x', index= False)
    except ValueError:
        print('file already exists')
    print("exported to csv")

In [ ]:
#query to get EDA and training data 
select_cols = """date, CONCAT(fullVisitorId, visitId) AS unique_id, totals, trafficSource, device, geoNetwork, product, promotion, hits, eventInfo.eventAction AS event_action, 
        CASE
        WHEN eventInfo.eventAction = 'Add to Cart' OR eCommerceAction.action_type = '3' THEN 1
        ELSE 0
        END AS is_addtocart"""
oct = preprocess(2016, 10, select_cols, 'product', 'promotion', 'hits', 'trafficSource', 'device', 'geoNetwork','totals')

In [88]:
#select statement for validation
select_final = """date, CONCAT(fullVisitorId, visitId) AS unique_id, product.productListName,
product.localProductPrice,
eventInfo.eventCategory,
type,
eCommerceAction.step,
product.productPrice,
totals.pageviews,
page.pagePath,
appInfo.screenName,
CASE
    WHEN eventInfo.eventAction = 'Add to Cart' OR eCommerceAction.action_type = '3' THEN 1
    ELSE 0
    END AS is_addtocart"""

In [137]:
#feature engineering and cleaning of validation data 
def kfeature_eng(data):
    #encoding pagepath
    data['path_depth'] = [len(s.split('/'))-1 for s in data['pagePath']]
    #renaming
    data = data.drop(['pagePath', 'screenName', 'eventCategory','step'], axis =1)
    #impute
    data.loc[:, 'localProductPrice'] = data['localProductPrice'].fillna(data['localProductPrice'].median())
    data.loc[:,'productPrice'] = data['productPrice'].fillna(data['productPrice'].median())
    data.loc[:,'pageviews'] = data['pageviews'].fillna(data['pageviews'].median())
    data['eventinfo.eventCategory'] = data['eventinfo.eventCategory'].fillna(data['eventinfo.eventCategory'].mode())
    data = data.drop(['date', 'unique_id'], axis = 1)
    data = data.fillna(data.mode())
    data = data.drop_duplicates()
    data = data.reset_index().drop('index', axis =1)
    return(data)

In [125]:
mar1 = get_month_data(2017, 3, select_final)
test_mar = kfeature_eng(mar1)
export(mar_test, 'kf_test_317')